In [1]:
using CSV
using DataFrames

using JuMP
using Gurobi

using LinearAlgebra
using Statistics
using Random
Random.seed!(42)  # For reproducibility

TaskLocalRNG()

## Import data

In [2]:
# Paths (relative to notebook structure)
restaurant_path    = "../clean_data/restaurant_data_expanded.csv"
scrap_path         = "../clean_data/food_scrap_locations.csv"
neighborhood_path  = "../clean_data/neighborhood_supply.csv"

# Read CSVs into DataFrames
food_scrap_locations  = CSV.read(scrap_path, DataFrame)
neighborhood_supply   = CSV.read(neighborhood_path, DataFrame)
restaurant_data       = CSV.read(restaurant_path, DataFrame)
restaurant_data = restaurant_data[shuffle(1:nrow(restaurant_data))[1:500], :]

# Verify
println("Sampled restaurants: ", nrow(restaurant_data))
println("Total supply: ", sum(restaurant_data.waste) / 1e6, " million lbs")

Sampled restaurants: 500
Total supply: 37.292250259216736 million lbs


# Clean the data:

## Ensure no commas in numbers, each field cast as correct data type, drop unnecessary columns, and no negative supply from synthetic data

In [3]:
# ================================
# 3. CLEAN RESTAURANT
# ================================
# Columns: latitude | longitude | waste
rename!(restaurant_data, names(restaurant_data)[3] => :supply)

# Ensure Float64
restaurant_data.supply    = Float64.(restaurant_data.supply)
restaurant_data.latitude  = Float64.(restaurant_data.latitude)
restaurant_data.longitude = Float64.(restaurant_data.longitude)

# *** DROP RESTAURANTS WITH NEGATIVE SUPPLY ***
filter!(row -> row.supply >= 0, restaurant_data)


# ================================
# 3. CLEAN FOOD SCRAP CENTER DATA
# ================================
# rename Latitude and Longitude to latitude and longitude for consistency
rename!(food_scrap_locations, names(food_scrap_locations)[15] => :latitude)
rename!(food_scrap_locations, names(food_scrap_locations)[16] => :longitude)

# Keep only coordinates we need
food_scrap_locations.latitude = Float64.(food_scrap_locations.latitude)
food_scrap_locations.longitude = Float64.(food_scrap_locations.longitude)

# Keep only coordinate columns in food_scrap_locations
select!(food_scrap_locations, [:latitude, :longitude])

# ================================
# 4. CLEAN NEIGHBORHOOD SUPPLY DATA
# ================================
# Rename demand column for clarity
rename!(neighborhood_supply, names(neighborhood_supply)[4] => :supply_gap)

# Now neighborhood_supply.supply_gap might be String OR Float64.
# Only do replace/parse if it's strings.
if eltype(neighborhood_supply.supply_gap) <: AbstractString
    neighborhood_supply.supply_gap =
        parse.(Float64, replace.(neighborhood_supply.supply_gap, "," => ""))
end

# Demand = positive deficit, surplus -> 0
neighborhood_supply.demand = max.(0.0, -neighborhood_supply.supply_gap)

neighborhood_supply.latitude  = Float64.(neighborhood_supply.latitude)
neighborhood_supply.longitude = Float64.(neighborhood_supply.longitude)

# keep only necessary columns from neighborhood supply
select!(neighborhood_supply, [:latitude, :longitude, :demand, :Year])

# ================================
# 5. SHOW CLEANED HEADS
# ================================
println("=== Restaurants (cleaned) ===")
println(first(restaurant_data, 5))

println("\n=== Food Scrap Locations (cleaned) ===")
println(first(food_scrap_locations, 5))

println("\n=== Neighborhood Supply (cleaned) ===")
println(first(neighborhood_supply, 5))

=== Restaurants (cleaned) ===
5×3 DataFrame
 Row │ latitude  longitude  supply        
     │ Float64   Float64    Float64       
─────┼────────────────────────────────────
   1 │  40.7424   -73.9943      1.11117e5
   2 │  40.7323   -73.8722  71532.9
   3 │  40.7794   -73.9492  98561.5
   4 │  40.6102   -73.9208  19103.5
   5 │  40.7315   -73.861   94690.4

=== Food Scrap Locations (cleaned) ===
5×2 DataFrame
 Row │ latitude  longitude 
     │ Float64   Float64   
─────┼─────────────────────
   1 │  40.6355   -74.0228
   2 │  40.7526   -73.969
   3 │  40.7635   -74.0002
   4 │  40.762    -73.9693
   5 │  40.7174   -74.0108

=== Neighborhood Supply (cleaned) ===
5×4 DataFrame
 Row │ latitude  longitude  demand     Year  
     │ Float64   Float64    Float64    Int64 
─────┼───────────────────────────────────────
   1 │  40.8267   -73.9217  1.02143e5   2025
   2 │  40.8321   -73.8887  3.33493e5   2025
   3 │  40.8016   -73.9374  0.0         2025
   4 │  40.8469   -73.8918  1.13653e5   202

## Get vectors for supply[i] for all restaurants, demand[k] for all neighborhoods, and cij[i,j] and cjk[j,k] to plug directly into JuMP model

In [4]:
# ================
# 6. EXTRACT VECTORS - CORRECTED
# ================
R = nrow(restaurant_data)
D = nrow(food_scrap_locations)

supply = restaurant_data.supply              # s_i
demand = neighborhood_supply.demand          # d_k

# Get unique neighborhoods (spatial locations only, not duplicated across years)
neighborhood_locations = unique(
    neighborhood_supply[neighborhood_supply.Year .== 2023, :],
    [:latitude, :longitude]
)

N = nrow(neighborhood_locations)  # This should be 197, not 591

println("Problem dimensions:")
println("  Restaurants (R): $R")
println("  Distribution Centers (D): $D")
println("  Neighborhoods (N): $N")

# Prepare demand vectors (already correct from your earlier code)
demand_2023 = neighborhood_supply[neighborhood_supply.Year .== 2023, :].demand
demand_2024 = neighborhood_supply[neighborhood_supply.Year .== 2024, :].demand
demand_2025 = neighborhood_supply[neighborhood_supply.Year .== 2025, :].demand

demand_df = DataFrame(
    y2023 = demand_2023,
    y2024 = demand_2024,
    y2025 = demand_2025
)

# Total capacity
Q = sum(restaurant_data.supply)
F = 10000

# ================
# 7. MANHATTAN DISTANCE FUNCTION
# ================
manhattan(lat1, lon1, lat2, lon2) = abs(lat1 - lat2) + abs(lon1 - lon2)

# ================
# 8. COST MATRICES - CORRECTED
# ================
# c_ij: Restaurants (i) → Distribution centers (j)
c_ij = [manhattan(restaurant_data.latitude[i], restaurant_data.longitude[i],
                 food_scrap_locations.latitude[j], food_scrap_locations.longitude[j])
       for i in 1:R, j in 1:D]

# c_jk: Distribution centers (j) → Neighborhoods (k)
# Use neighborhood_locations (197 unique locations) instead of neighborhood_supply (591 rows)
c_jk = [manhattan(food_scrap_locations.latitude[j], food_scrap_locations.longitude[j],
                 neighborhood_locations.latitude[k], neighborhood_locations.longitude[k])
       for j in 1:D, k in 1:N]

println("\nCost matrix dimensions:")
println("  c_ij (R x D): ", size(c_ij))
println("  c_jk (D x N): ", size(c_jk))

# Verify everything matches
@assert size(c_ij) == (R, D) "c_ij dimension mismatch!"
@assert size(c_jk) == (D, N) "c_jk dimension mismatch!"
@assert nrow(demand_df) == N "demand_df rows must equal N!"
@assert N == 197 "N should be 197 (unique neighborhoods), not 591!"

println("\n✓ All dimensions verified!")
println("  N = $N")
println("  demand_df rows = $(nrow(demand_df))")
println("  c_jk columns = $(size(c_jk)[2])")

Problem dimensions:
  Restaurants (R): 500
  Distribution Centers (D): 201
  Neighborhoods (N): 197

Cost matrix dimensions:
  c_ij (R x D): (500, 201)
  c_jk (D x N): (201, 197)

✓ All dimensions verified!
  N = 197
  demand_df rows = 197
  c_jk columns = 197


# Sanity check data

## Stochastic Demand

### Adaptive Stochastic: Opening Centers Based On Historical Demand Distribution

We formulate the problem as a two-stage stochastic program with recourse. We solve the multi-objective problem using the weighted-sum method, systematically varying weights $w_c$ and $w_e$ to trace the Pareto frontier.

**Sets and Indices:**
- $R$: Set of restaurants, indexed by $i$
- $D$: Set of distribution centers, indexed by $j$
- $N$: Set of neighborhoods, indexed by $k$
- $S$: Set of demand scenarios, indexed by $s \in \{2023, 2024, 2025\}$

**Parameters:**
- $s_i$: Food supply available at restaurant $i$
- $d_k^s$: Food demand in neighborhood $k$ under scenario $s$
- $c_{ij}$: Transportation cost from restaurant $i$ to center $j$
- $c_{jk}$: Transportation cost from center $j$ to neighborhood $k$
- $F$: Fixed cost of opening a distribution center
- $Q$: Total system capacity (sum of all supply)
- $p_s$: Probability of scenario $s$ occurring

**Decision Variables:**
- $z_j \in \{0,1\}$: Binary variable indicating if center $j$ is opened (first-stage)
- $x_{ij}^s \geq 0$: Flow from restaurant $i$ to center $j$ in scenario $s$
- $y_{jk}^s \geq 0$: Flow from center $j$ to neighborhood $k$ in scenario $s$
- $u_k^s \geq 0$: Unmet demand at neighborhood $k$ in scenario $s$
- $t^s \geq 0$: Worst (maximum) unmet demand across all neighborhoods in scenario $s$

**Objective Function:**

$$\min \quad F \sum_{j \in D} z_j + \sum_{s \in S} p_s \left[ w_c \cdot \text{Cost}^s + w_e \cdot t^s \right]$$

where:

$$\text{Cost}^s = \sum_{i \in R, j \in D} c_{ij} x_{ij}^s + \sum_{j \in D, k \in N} c_{jk} y_{jk}^s$$

**Constraints:**

1. **Supply constraints:** Each restaurant must send all its supply
   $$\sum_{j \in D} x_{ij}^s = s_i \quad \forall i \in R, s \in S$$

2. **Flow conservation:** What flows into a center must flow out
   $$\sum_{k \in N} y_{jk}^s = \sum_{i \in R} x_{ij}^s \quad \forall j \in D, s \in S$$

3. **Demand satisfaction:** Demand is met by deliveries plus unmet demand
   $$\sum_{j \in D} y_{jk}^s + u_k^s = d_k^s \quad \forall k \in N, s \in S$$

4. **Center capacity:** Flow through a center only if it's open
   $$\sum_{i \in R} x_{ij}^s \leq Q \cdot z_j \quad \forall j \in D, s \in S$$

5. **Equity constraint (minimax):** Track worst unmet demand in each scenario
   $$t^s \geq u_k^s \quad \forall k \in N, s \in S$$

In [5]:
R = nrow(restaurant_data)
D = nrow(food_scrap_locations)
N = nrow(neighborhood_supply[neighborhood_supply.Year .== 2023, :])
Q = sum(restaurant_data.supply)

M = sum(demand)
F = 10000       # Center opening cost

S = 3 # 3 demand years

demand_2023 = neighborhood_supply[neighborhood_supply.Year .== 2023, :].demand
demand_2024 = neighborhood_supply[neighborhood_supply.Year .== 2024, :].demand
demand_2025 = neighborhood_supply[neighborhood_supply.Year .== 2025, :].demand

demand_df = DataFrame(
    y2023 = demand_2023,
    y2024 = demand_2024,
    y2025 = demand_2025
)

Row,y2023,y2024,y2025
,Float64,Float64,Float64
1,0.0,0.0,1.02143e5
2,0.0,0.0,3.33493e5
3,0.0,0.0,0.0
4,0.0,0.0,1.13653e5
5,0.0,0.0,0.0
6,82217.3,13128.3,5.06273e5
7,2.25651e5,0.0,0.0
8,9.04474e5,0.0,1.13911e5
9,0.0,0.0,0.0


In [6]:
function solve_model(
    restaurant_data, food_scrap_locations, demand_df, cij, cjk,
    w_cost::Float64, w_equity::Float64;
    F::Float64 = 10000.0,
    time_limit::Int = 60
)
    
    R = nrow(restaurant_data)
    D = nrow(food_scrap_locations)
    N = nrow(demand_df)
    S = 3
    p = 1/3
    Q = sum(restaurant_data.supply)
    
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", time_limit)
    set_optimizer_attribute(model, "MIPGap", 0.01)
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    @variable(model, z[1:D], Bin)
    @variable(model, x[1:R, 1:D, 1:S] >= 0)
    @variable(model, y[1:D, 1:N, 1:S] >= 0)
    @variable(model, u[1:N, 1:S] >= 0)
    @variable(model, t[1:S] >= 0)
    
    @expression(model, transport[s in 1:S],
        sum(cij[i,j] * x[i,j,s] for i in 1:R, j in 1:D) +
        sum(cjk[j,k] * y[j,k,s] for j in 1:D, k in 1:N)
    )
    
    @objective(model, Min,
        F * sum(z) + p * sum(w_cost * transport[s] + w_equity * t[s] for s in 1:S)
    )
    
    @constraint(model, [i in 1:R, s in 1:S], sum(x[i,j,s] for j in 1:D) == restaurant_data.supply[i])
    @constraint(model, [j in 1:D, s in 1:S], sum(y[j,k,s] for k in 1:N) == sum(x[i,j,s] for i in 1:R))
    @constraint(model, [k in 1:N], sum(y[j,k,1] for j in 1:D) + u[k,1] == demand_df.y2023[k])
    @constraint(model, [k in 1:N], sum(y[j,k,2] for j in 1:D) + u[k,2] == demand_df.y2024[k])
    @constraint(model, [k in 1:N], sum(y[j,k,3] for j in 1:D) + u[k,3] == demand_df.y2025[k])
    @constraint(model, [j in 1:D, s in 1:S], sum(x[i,j,s] for i in 1:R) <= Q * z[j])
    @constraint(model, [k in 1:N, s in 1:S], t[s] >= u[k,s])
    
    optimize!(model)
    
    if termination_status(model) in [MOI.OPTIMAL, MOI.TIME_LIMIT, MOI.ALMOST_OPTIMAL]
        worst = [value(t[s]) for s in 1:S]
        return (
            model = model,   
            x = x,           
            y = y,            
            u = u,            
            t = t,            
            z = z,            
            w_cost = w_cost,
            w_equity = w_equity,
            centers = sum(value.(z) .> 0.5),
            transport = p * sum(value(transport[s]) for s in 1:S),
            equity = p * sum(worst),
            worst_2023 = worst[1],
            worst_2024 = worst[2],
            worst_2025 = worst[3],
            total_unmet_2023 = sum(value(u[k,1]) for k in 1:N),
            total_unmet_2024 = sum(value(u[k,2]) for k in 1:N),
            total_unmet_2025 = sum(value(u[k,3]) for k in 1:N)
        )
    else
        return nothing
    end
end

solve_model (generic function with 1 method)

In [7]:
r = solve_model(restaurant_data, food_scrap_locations, demand_df, c_ij, c_jk,
                1.0, 2.0; time_limit=30)
println("  Centers: $(r.centers), Avg worst unmet: $(round(mean([r.worst_2023, r.worst_2024, r.worst_2025])/1e3, digits=1))k")

Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter TimeLimit to value 30
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 30
  Centers: 135, Avg worst unmet: 524.9k


# Extract the data needed for visualization

In [8]:
# ============================================================================
# 1. Export Location Data
# ============================================================================
CSV.write("viz_data/restaurants.csv", DataFrame(
    id = 1:R,
    latitude = restaurant_data.latitude,
    longitude = restaurant_data.longitude,
    supply = restaurant_data.supply
))

CSV.write("viz_data/donation_centers.csv", DataFrame(
    id = 1:D,
    latitude = food_scrap_locations.latitude,
    longitude = food_scrap_locations.longitude
))

# Update the neighborhoods export to include names
# Reload original neighborhood data to get names
neighborhood_full = CSV.read(neighborhood_path, DataFrame)
nbhd_2023_full = neighborhood_full[neighborhood_full.Year .== 2023, :]

# Export neighborhoods with names
CSV.write("viz_data/neighborhoods.csv", DataFrame(
    id = 1:N,
    name = nbhd_2023_full[!, "Neighborhood.Tabulation.Area..NTA..Name"],
    latitude = nbhd_2023_full.latitude,
    longitude = nbhd_2023_full.longitude,
    demand_2023 = demand_df.y2023,
    demand_2024 = demand_df.y2024,
    demand_2025 = demand_df.y2025
))

"viz_data/neighborhoods.csv"

In [9]:
# ============================================================================
# 2. Export Scenario-Specific Results for a Balanced Solution
# ============================================================================

R = nrow(restaurant_data)
D = nrow(food_scrap_locations)
N = nrow(demand_df)
S = 3

# Solve and get variables
result_bal = solve_model(restaurant_data, food_scrap_locations, demand_df, c_ij, c_jk, 1.0, 2.0; time_limit=60)

if !isnothing(result_bal)
    # Extract variables
    z_bal = result_bal.z
    x_bal = result_bal.x
    y_bal = result_bal.y
    u_bal = result_bal.u
    
    # Centers opened
    CSV.write("viz_data/centers_opened.csv", DataFrame(
        id = 1:D,
        opened = Int.(round.(value.(z_bal)))
    ))
    
    # Per-scenario allocations
    for s in 1:S
        year = 2022 + s
        r_bal = [sum(value(y_bal[j,k,s]) for j in 1:D) for k in 1:N]
        
        CSV.write("viz_data/allocations_$year.csv", DataFrame(
            neighborhood_id = 1:N,
            received = r_bal,
            unmet = value.(u_bal[:,s])
        ))
    end
    
    # Flows for scenario 1 (2023)
    flows = DataFrame(from_type=String[], from_id=Int[], to_type=String[], to_id=Int[], flow=Float64[])
    for i in 1:R, j in 1:D
        value(x_bal[i,j,1]) > 1e-3 && push!(flows, ("restaurant", i, "center", j, value(x_bal[i,j,1])))
    end
    for j in 1:D, k in 1:N
        value(y_bal[j,k,1]) > 1e-3 && push!(flows, ("center", j, "neighborhood", k, value(y_bal[j,k,1])))
    end
    CSV.write("viz_data/flows_2023.csv", flows)
end

# ============================================================================
# 3. Export Scenario Comparison Data
# ============================================================================
scenario_summary = DataFrame(
    scenario = ["2023", "2024", "2025"],
    total_demand = [sum(demand_df.y2023), sum(demand_df.y2024), sum(demand_df.y2025)],
    total_received = [sum(sum(value(y_bal[j,k,s]) for j in 1:D) for k in 1:N) for s in 1:S],
    total_unmet = [sum(value(u_bal[k,s]) for k in 1:N) for s in 1:S],
    worst_unmet = [value(result_bal.t[s]) for s in 1:S]
)
CSV.write("viz_data/scenario_summary.csv", scenario_summary)

Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 60


"viz_data/scenario_summary.csv"

In [10]:
# ============================================================================
# 4. Generate Pareto Frontier Data AND Export Allocations
# ============================================================================

# Weight pairs for equity range 1-10
weight_pairs = [
    (1.0, 0.1),   # cost-focused
    (1.0, 0.5),   # cost-focused
    (1.0, 1.0),   # balanced
    (1.0, 2.0),
    (1.0, 3.0),
    (1.0, 4.0),
    (1.0, 5.0),
    (1.0, 6.0),
    (1.0, 7.0)
]

results = DataFrame(
    w_cost = Float64[],
    w_eq = Float64[],
    obj_value = Float64[],
    num_centers = Int[],
    avg_transport_cost = Float64[],
    avg_equity_t = Float64[],
    avg_total_recv = Float64[],
    avg_total_unmet = Float64[],
    t_2023 = Float64[],
    t_2024 = Float64[],
    t_2025 = Float64[]
)

# Create output directories
mkpath("viz_data")
mkpath("viz_data/pareto_allocations")

N = nrow(demand_df)
S = 3

println("Generating Pareto frontier with $(length(weight_pairs)) points...")

for (i, (w_cost, w_eq)) in enumerate(weight_pairs)
    println("[$i/$(length(weight_pairs))] w_cost=$w_cost, w_eq=$w_eq")
    
    result = solve_model(
        restaurant_data, food_scrap_locations, demand_df, c_ij, c_jk,
        w_cost, w_eq;
        time_limit = 60
    )
    
    if !isnothing(result)
        # Add to Pareto results
        push!(results, (
            result.w_cost,
            result.w_equity,
            result.w_cost * result.transport + result.w_equity * result.equity,  # Objective value
            result.centers,
            result.transport,
            result.equity,
            (result.total_unmet_2023 + result.total_unmet_2024 + result.total_unmet_2025) / 3,  # avg total unmet
            (result.total_unmet_2023 + result.total_unmet_2024 + result.total_unmet_2025) / 3,
            result.worst_2023,
            result.worst_2024,
            result.worst_2025
        ))
        
        # Export allocations for this solution
        u = result.u
        avg_unmet = [mean([value(u[k,s]) for s in 1:S]) for k in 1:N]
        
        CSV.write("viz_data/pareto_allocations/alloc_$(i).csv", DataFrame(
            neighborhood_id = 1:N,
            avg_unmet = avg_unmet,
            w_cost = w_cost,
            w_eq = w_eq,
            centers_opened = result.centers
        ))
        
        println("  ✓ Centers: $(result.centers), Avg equity: $(round(mean([result.worst_2023, result.worst_2024, result.worst_2025])/1e3, digits=1))k")
    else
        println("  ✗ Failed to solve")
    end
end

# Save Pareto results
CSV.write("viz_data/pareto_results.csv", results)

println("\nPareto frontier complete! $(nrow(results)) solutions found.")
println("Saved to viz_data/pareto_results.csv")
println("Saved allocations to viz_data/pareto_allocations/")

Generating Pareto frontier with 9 points...
[1/9] w_cost=1.0, w_eq=0.1
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 60
  ✓ Centers: 110, Avg equity: 2067.5k
[2/9] w_cost=1.0, w_eq=0.5
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 60
  ✓ Centers: 113, Avg equity: 1058.7k
[3/9] w_cost=1.0, w_eq=1.0
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter